In [ ]:
!pip install tensorflow==2.6.2 -U

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_name = 'amazon-redshift-police-generate-image'
tag = ':1'
%cd ./docker/make_image_container
# !docker rmi -f $(docker images -a -q)
!docker build -t {image_name}{tag} .
%cd ../../

In [ ]:
!sudo rm -rf img/
!sudo rm -rf data/

In [ ]:
!docker run --rm -v $(pwd):/workspace {image_name}{tag} python3 src/generate_image.py --check-names "Amazon Redshift/Amazon RedShift"

In [ ]:
!rm img.zip
!zip -r img.zip img/

In [ ]:
image_name = 'amazon-redshift-police-train-image'
tag = ':1'
%cd ./docker/train_container
# !docker rmi -f $(docker images -a -q)
!docker build -t {image_name}{tag} .
%cd ../../

In [ ]:
!docker run --rm -v $(pwd):/workspace {image_name}{tag} python3 src/train.py --sm-model-dir "model" --train "data" --epochs 1

In [ ]:
model = tf.keras.models.load_model('./model/000000001/')

In [ ]:
train_X = np.load('data/train_X.npy')
train_y = np.load('data/train_y.npy')

In [ ]:
for i in range(train_X.shape[0]):
    pred_y = 0 if model.predict(train_X[i:i+1,:,:]) < 0.5 else 1
    if pred_y==train_y[i]:
        print('OK')
    else:
        print('NG')

In [ ]:
plt.plot(pred_y,train_y)